In [37]:
import pandas as pd
import talib

In [38]:
xbt_df_15m = pd.read_csv("../data/XBTUSD-15m-data.csv")

In [39]:
xbt_df_15m

,timestamp,symbol,open,high,low,close,volume
0,2015-09-25 12:15:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0
1,2015-09-25 12:30:00+00:00,XBTUSD,NaN,NaN,NaN,NaN,0
2,2015-09-25 12:45:00+00:00,XBTUSD,239.99,239.99,237.37,237.49,6501
3,2015-09-25 13:00:00+00:00,XBTUSD,237.49,237.47,237.36,237.45,5000
4,2015-09-25 13:15:00+00:00,XBTUSD,237.45,237.45,237.45,237.45,0
...,...,...,...,...,...,...,...
165802,2020-06-17 14:45:00+00:00,XBTUSD,9403.00,9419.00,9352.00,9395.00,73757898
165803,2020-06-17 15:00:00+00:00,XBTUSD,9395.00,9415.00,9390.00,9401.00,18192646
165804,2020-06-17 15:15:00+00:00,XBTUSD,9401.00,9414.50,9390.50,9411.00,18987626
165805,2020-06-17 15:30:00+00:00,XBTUSD,9411.00,9416.00,9401.50,9411.00,8455552


In [40]:
# calculating
ema_slow = talib.EMA(xbt_df_15m['close'], timeperiod=188)
ema_fast = talib.EMA(xbt_df_15m['close'], timeperiod=153)
adx = talib.ADX(high=xbt_df_15m['high'],low=xbt_df_15m['low'], close=xbt_df_15m['close'], timeperiod=6)

In [41]:
ema_fast
ema_slow
adx

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
165802    31.634760
165803    33.598039
165804    35.234105
165805    36.107864
165806    38.111101
Length: 165807, dtype: float64

In [42]:
in_position = False
adx_threshold = 28.57878459432971
trailing = 0.6056322493449964  # %

In [43]:
entry_price = 0
stop_price = 0
run_up_price = 0
dd_price = 0
start = None
end = None
order_history = []
for index, row in xbt_df_15m.iterrows():
    price = row['close']
    is_trending_market = adx[index] >= adx_threshold
    long_condition = not in_position and is_trending_market and ema_fast[index-1] <= ema_slow[index-1] and ema_fast[index] > ema_slow[index] and price > ema_slow[index]
    exit_long_condition = in_position and ema_fast[index] < ema_slow[index] and price < ema_slow[index]
    stop_long_condition = False
    if long_condition:
        in_position = True
        entry_price = price
        run_up_price = price
        dd_price = price
        start = row['timestamp']
    if in_position:
        # re-calculate run_up and dd price and stop_price
        run_up_price = max(price, run_up_price)
        dd_price = min(price, dd_price)
        stop_price = run_up_price * (1 - trailing/100)
        stop_long_condition = price <= stop_price

    if stop_long_condition or exit_long_condition:
        in_position = False
        end = row['timestamp']
        order_type = "stop" if stop_long_condition else "exit"
        profit = 100*(price - entry_price)/entry_price
        order_history.append({"start": start, "end": end, "type": order_type, "entry": entry_price, "exit": price, "profit": profit, "dd": dd_price, "stop": stop_price, "ru": run_up_price})
        # reset
        entry_price = 0
        run_up_price = 0
        dd_price = 0
        stop_price = 0
        start = None
        end = None

In [44]:
order_history_df = pd.DataFrame(order_history)
order_history_df.to_csv("XBTUSD-15m-results.csv")